In [47]:
import pandas as pd
import seaborn as sns
import bs4
import requests
import os
from scipy.misc import imresize
from bleedml.classifiers import CascadeForest
from bleedml.utils import multi2Dscan, scan2D
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.ensemble import RandomForestClassifier
from importlib import reload
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [33]:
def get_image_links(search, limit=100):
    link = 'https://www.google.com/search?q={search}&oq={search}&start={start}'
    start = 0
    links = []
    while len(links) < limit:
        print(start)
        page = requests.get(link.format(search=search, start=0))
        if page.status_code == 200:
            soup = bs4.BeautifulSoup(page.text, 'html5lib')
            newlinks = [i.get('src') for i in soup.find_all('img')]
            start += len(newlinks)
            links.extend(newlinks)
        else:
            print('error in link')
    return links

In [34]:
if not os.path.exists('wheatlinks'):
    links = get_image_links('wheat')
    links = list(set(links))
    with open('wheatlinks', 'w') as fl:
        fl.write('\n'.join(links))
    ! mkdir ../images/wheat/
    ! cd ../images/wheat/ && wget -i ../../notebooks/wheatlinks

0
12
24
36
48
60
72
84
96
mkdir: cannot create directory ‘../images/wheat/’: File exists
--2017-03-21 01:25:46--  https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA
Resolving encrypted-tbn1.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Connecting to encrypted-tbn1.gstatic.com|216.58.220.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4538 (4.4K) [image/jpeg]
Saving to: ‘images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA.1’

images?q=tbn:ANd9Gc 100%[===================>]   4.43K  --.-KB/s    in 0.005s  

2017-03-21 01:25:46 (956 KB/s) - ‘images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA.1’ saved [4538/4538]

--2017-03-21 01:25:46--  https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSgXGeSanMxeGkDxBJ6uyyXG-k4wLOFKJgxlhKap3TvCVKwpGcYg0rPqsWeSWJ24JJDuJ8
Resolving encrypted-tbn3.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Conn

In [35]:
if not os.path.exists('ricelinks'):
    links = get_image_links('rice')
    links = list(set(links))
    with open('ricelinks', 'w') as fl:
        fl.write('\n'.join(links))

    ! mkdir ../images/rice/
    ! cd ../images/rice/ && wget -i ../../notebooks/ricelinks

0
14
28
42
56
70
84
98
../../notebooks/ricelinks: Invalid URL //lh6.googleusercontent.com/proxy/5yZ3VhEXaYVuQcRPhySuxoWU92ojkm2xYQx_oB--JSi05yGnblcToVRjOYjmpFMmgoVvYBBgqvTVqBUqv5lLl4Hjh9oBCX6K0Pf20mVRdId8Al08zt3fUO87NJRWuP__HGPl3tUEeo9aQjsXfUtArmbNz8Ozdg=w115-h115: Scheme missing
--2017-03-21 01:26:10--  https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSnVe-MQhPVgzn2Cz0NLmK-wZIRvRkktysAIDH2oaITE7ddfX0XUS6ItA
Resolving encrypted-tbn1.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Connecting to encrypted-tbn1.gstatic.com|216.58.220.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341 (3.3K) [image/jpeg]
Saving to: ‘images?q=tbn:ANd9GcSnVe-MQhPVgzn2Cz0NLmK-wZIRvRkktysAIDH2oaITE7ddfX0XUS6ItA’

images?q=tbn:ANd9Gc 100%[===================>]   3.26K  --.-KB/s    in 0.001s  

2017-03-21 01:26:10 (5.31 MB/s) - ‘images?q=tbn:ANd9GcSnVe-MQhPVgzn2Cz0NLmK-wZIRvRkktysAIDH2oaITE7ddfX0XUS6ItA’ saved [3341/3341]

--2017-03-21 01:26:10--  https://encrypted-tb

### Creating training dataset

In [36]:
wheat_paths = []
for fl in os.listdir('../images/wheat/'):
    path = '../images/wheat/' + fl
    wheat_paths.append(path)

In [37]:
rice_paths = []
for fl in os.listdir('../images/rice/'):
    path = '../images/rice/' + fl
    barley_paths.append(path)

In [38]:
wx, wy = set(), set()
for path in wheat_paths:
    x, y, _  = imread(path).shape
    wx.add(x); wy.add(y)

In [39]:
for path in rice_paths:
    x, y, _  = imread(path).shape
    wx.add(x); wy.add(y)
wx = list(sorted(wx))
wy = list(sorted(wy))
wx[0], wy[0]

(55, 72)

In [40]:
images, labels = [], []
for path in rice_paths:
    image = imresize(imread(path), (wx[0], wy[0]))
    images.append(image)
    labels.append('rice')
for path in wheat_paths:
    image = imresize(imread(path), (wx[0], wy[0]))
    images.append(image)
    labels.append('wheat')
mask = np.random.random(len(labels)) < 1
images = np.array(images)[mask]
labels = np.array(labels)[mask]

print(len(labels))

33


In [45]:
X, y = [i.flatten() for i in images], labels
scores = cross_val_score(RandomForestClassifier(n_jobs=-1), X, y, cv=3)
print(scores)

[ 1.  1.  1.]


In [46]:
est = RandomForestClassifier(n_jobs=-1)
est.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [48]:
with open('../image_classifier.pickle', 'wb') as fl:
    pickle.dump(est, fl)